In [1]:
import os
import subprocess
import multiprocessing

In [3]:
def get_processor_count():
    return multiprocessing.cpu_count()

processor_count = get_processor_count()
print("Количество процессоров:", processor_count)

Количество процессоров: 6


In [ ]:
os.environ["GATK_LOCAL_JAR"] = "./utils/gatk/gatk-package-4.3.0.0-local.jar"
os.makedirs("./results", exist_ok=True)
input_dir = "./input"
log_dir = "./logs"
os.makedirs(log_dir, exist_ok=True)

panel = "./bed/hg38_solotest.interval_list"

In [ ]:
if not os.listdir(input_dir):
    print("Empty input directory")
    exit(1)
else:
    names = []
    for infile in os.listdir(input_dir):
        if infile.endswith("_L00?_R1.fq.gz"):
            file = os.path.join(input_dir, infile)
            base = os.path.splitext(infile)[0]
            name = base.split("_")[0]
            if name not in names:
                names.append(name)
    with open(os.path.join(log_dir, base + ".log"), "w") as logfile:
        logfile.write("names=" + repr(names) + "\n")
    for name in names:
        files = []
        lines = []
        bams = []
        with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
            logfile.write("Processing sample: " + name + "\n")
            files = [os.path.join(input_dir, f"{name}_L00{line}_R1.fq.gz") for line in lines]
            logfile.write("files=" + repr(files) + "\n")
            for infile in files:
                base = os.path.splitext(os.path.basename(infile))[0]
                line = base.split("_")[1]
                R1 = f"{name}_{line}_R1.fq.gz"
                R2 = f"{name}_{line}_R2.fq.gz"
                with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
                    logfile.write("Processing line: " + line + "\n")
                    logfile.write("Processing pair: " + fR1 + " " + fR2 + "\n")
                    logfile.write("Adapter trimming...\n")
                trimmomatic_dir = "./results/trimmomatic"
                os.makedirs(trimmomatic_dir, exist_ok=True)
                command = [
                    "java",
                    "-jar",
                    "./utils/Trimmomatic/trimmomatic-0.39.jar",
                    "PE",
                    "-threads",
                    str(processor_count),
                    "-phred33",
                    os.path.join(input_dir, R1),
                    os.path.join(input_dir, R2),
                    os.path.join(trimmomatic_dir, base + "_R1.trim.fastq.gz"),
                    os.path.join(trimmomatic_dir, base + "_R1un.trim.fastq.gz"),
                    os.path.join(trimmomatic_dir, base + "_R2.trim.fastq.gz"),
                    os.path.join(trimmomatic_dir, base + "_R2un.trim.fastq.gz"),
                    "ILLUMINACLIP:./utils/Trimmomatic/TruSeq3-PE-2.fa:2:40:15:2:true",
                    "LEADING:3",
                    "TRAILING:3",
                    "SLIDINGWINDOW:6:15",
                    "MINLEN:36",
                ]
                with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
                    logfile.write("Converting to uBAM...\n")
                subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT, text=True)
                
                with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
                    logfile.write("Aligning...\n")
                    samtofastq_command = [
                        "./utils/gatk/gatk",
                        "--java-options",
                        "-XX:-PrintFlagsFinal -XX:+UseG1GC -Dsamjdk.use_async_io_read_samtools=true -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10 -Xms1000m -Xmx16000m",
                        "SamToFastq",
                        "--INPUT",
                        os.path.join("./results/FastqToSam", base + ".unmapped.qsorted.bam"),
                        "--FASTQ",
                        "/dev/stdout",
                        "--INTERLEAVE",
                        "true",
                        "--NON_PF",
                        "true",
                    ]
                    bwa_command = [
                        "bwa",
                        "mem",
                        "-K",
                        "100000000",
                        "-c",
                        "250",
                        "-p",
                        "-v",
                        "3",
                        "-t",
                        str(processor_count),
                        "-Y",
                        "./utils/bwa/GRCh38.fa",
                        "/dev/stdin",
                        "-",
                    ]
                    mergebamalignment_command = [
                        "./utils/gatk/gatk",
                        "--java-options",
                        "-XX:-PrintFlagsFinal -XX:+UseG1GC -Dsamjdk.compression_level=5 -Dsamjdk.use_async_io_read_samtools=true -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10 -Xms1000m -Xmx16000m",
                        "MergeBamAlignment",
                        "--VALIDATION_STRINGENCY",
                        "SILENT",
                        "--EXPECTED_ORIENTATIONS",
                        "FR",
                        "--ATTRIBUTES_TO_RETAIN",
                        "X0",
                        "--ATTRIBUTES_TO_REMOVE",
                        "NM",
                        "--ATTRIBUTES_TO_REMOVE",
                        "MD",
                        "--ALIGNED_BAM",
                        "/dev/stdin",
                        "--UNMAPPED_BAM",
                        os.path.join("./results/FastqToSam", base + ".unmapped.qsorted.bam"),
                        "--OUTPUT",
                        os.path.join("./results/BWA", base + ".aligned.sorted.bam"),
                        "--REFERENCE_SEQUENCE",
                        "./utils/ref/GRCh38.fa",
                        "--PAIRED_RUN",
                        "true",
                        "--SORT_ORDER",
                        "coordinate",
                        "--IS_BISULFITE_SEQUENCE",
                        "false",
                        "--ALIGNED_READS_ONLY",
                        "false",
                        "--CLIP_ADAPTERS",
                        "false",
                        "--MAX_RECORDS_IN_RAM",
                        "5000000",
                        "--ADD_MATE_CIGAR",
                        "true",
                        "--MAX_INSERTIONS_OR_DELETIONS",
                        "-1",
                        "--PRIMARY_ALIGNMENT_STRATEGY",
                        "MostDistant",
                        "--PROGRAM_RECORD_ID",
                        "bwamem",
                        "--PROGRAM_GROUP_VERSION",
                        "0.7.17",
                        "--PROGRAM_GROUP_COMMAND_LINE",
                        "bwa mem -K 100000000 -M -c 250 -p -v 3 -t " + str(processor_count) + " -Y ./utils/bwa/GRCh38.fa",
                        "--PROGRAM_GROUP_NAME",
                        "bwamem",
                        "--UNMAPPED_READ_STRATEGY",
                        "COPY_TO_TAG",
                        "--ALIGNER_PROPER_PAIR_FLAGS",
                        "true",
                        "--UNMAP_CONTAMINANT_READS",
                        "true",
                        "--CREATE_INDEX",
                        "true",
                        "--ADD_PG_TAG_TO_READS",
                        "false",
                    ]
                    
                subprocess.run(samtofastq_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                samtofastq_output = subprocess.Popen(
                    bwa_command,
                    stdin=subprocess.PIPE,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True,
                )
                mergebamalignment_output = subprocess.run(
                    mergebamalignment_command,
                    stdin=samtofastq_output.stdout,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    text=True,
                )
                samtofastq_output.stdout.close()
                    
                bams += "-I ./results/BWA/" + base + ".aligned.sorted.bam "
                    
                os.makedirs(os.path.join(results_dir, "FastqToSam"), exist_ok=True)
                os.makedirs(os.path.join(results_dir, "BQSR"), exist_ok=True)
                os.makedirs(os.path.join(results_dir, "GATK"), exist_ok=True)
                ampliconmetrics_dir = "./ampliconmetrics"
                os.makedirs(ampliconmetrics_dir, exist_ok=True)
                cover_dir = "./cover"
                os.makedirs(cover_dir, exist_ok=True)
                    
                with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
                    logfile.write("Variants recalibrations...\n")
                    bqsrt_command = [
                        "./utils/gatk/gatk",
                        "--java-options",
                        "-XX:-PrintFlagsFinal -XX:+UseParallelGC -Dsamjdk.compression_level=5 -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10 -Xms1000m -Xmx8000m",
                        "BaseRecalibrator",
                        "-R",
                        "./utils/ref/GRCh38.fa",
                        bams,
                        "--use-original-qualities",
                        "true",
                        "-O",
                        os.path.join(results_dir, "BQSR", name + ".stats"),
                        "--known-sites",
                        "./utils/ref/dbsnp_138.hg38.vcf.gz",
                        "--known-sites",
                        "./utils/ref/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz",
                        "-L",
                        panel,
                    ]
                    applybqsr_command = [
                        "./utils/gatk/gatk",
                        "--java-options",
                        "-XX:-PrintFlagsFinal -XX:+UseParallelGC -Dsamjdk.compression_level=5 -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10 -Xms1000m -Xmx8000m",
                        "ApplyBQSR",
                        "--create-output-bam-md5",
                        "--add-output-sam-program-record",
                        "true",
                        "-R",
                        "./utils/ref/GRCh38.fa",
                        bams,
                        "--use-original-qualities",
                        "true",
                        "-O",
                        os.path.join(results_dir, "BQSR", name + ".recalibrated.aligned.bam"),
                        "--create-output-bam-index",
                        "--bqsr-recal-file",
                        os.path.join(results_dir, "BQSR", name + ".stats"),
                        "--static-quantized-quals",
                        "10",
                        "--static-quantized-quals",
                        "20",
                        "--static-quantized-quals",
                        "30",
                        "-L",
                        panel,
                    ]
                with open(os.path.join(log_dir, base + ".log"), "a") as logfile:
                    logfile.write("Variant calling...\n")
                    haplotypecaller_command = [
                        "./utils/gatk/gatk",
                        "--java-options",
                        "-XX:-PrintFlagsFinal -XX:+UseParallelGC -XX:ParallelGCThreads=2 -Dsamjdk.use_async_io_read_samtools=true -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -XX:GCTimeLimit=50 -XX:GCHeapFreeLimit=10 -Xms1000m -Xmx8000m",
                        "HaplotypeCaller",
                        "-R",
                        "./utils/ref/GRCh38.fa",
                        "-I",
                        os.path.join(results_dir, "BQSR", name + ".recalibrated.aligned.bam"),
                        "-L",
                        panel,
                        "-O",
                        os.path.join(results_dir, "GATK", name + ".vcf.gz"),
                        "--max-alternate-alleles",
                        "6",
                        "--max-reads-per-alignment-start",
                        "0",
                        "--pair-hmm-implementation",
                        "FASTEST_AVAILABLE",
                        "--interval-padding",
                        "25",
                        "--native-pair-hmm-threads",
                        str(processor_count),
                        "--smith-waterman",
                        "JAVA",
                        "-A",
                        "Coverage",
                        "-A",
                        "ChromosomeCounts",
                        "-A",
                        "BaseQuality",
                        "-A",
                        "FragmentLength",
                        "-A",
                        "BaseQualityRankSumTest",
                        "-A",
                        "MappingQuality",
                        "-A",
                        "ReadPosRankSumTest",
                        "-A",
                        "FisherStrand",
                        "-A",
                        "ClippingRankSumTest",
                        "-A",
                        "StrandOddsRatio",
                        "-A",
                        "ReadPosition",
                    ]
        subprocess.run(bqsrt_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        subprocess.run(applybqsr_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        subprocess.run(haplotypecaller_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)